In [1]:
##Allow Notebook to Import from Comp_Chem_Package
import sys
if("win" in sys.platform):
    sys.path.append(".\\Comp_Chem_Package")
else: 
    sys.path.append("./Comp_Chem_Package")

## Diatomic Workspace 
The Diatomic Workspace notebook is designed to allow it users to interact directly with the Computational Chemistry package without needing to enage in any programming. It is strictly a user interface that provides simple access to calculation of the wavefunctions for diatomic molecules. Please see the interface below, and provide the information requested into the widgets to begin using this interface. 

In [2]:
#Import all required classes here
from nistScraper import getDiatomicConstants
from how import *
from basisSet import *
from rkr import *
from morsePotential import *
from operators import *
from schrodinger import *
import dill
import pickle
import os

In [3]:
#Set up widgets here
diatomicName = widgets.Text(
    value = "CO",
    description = '<p style="font-family:verdana;font-size:15px">Diatomic</p>'
)
basisSize = widgets.IntText(
    value = 4,
    description = '<p style="font-family:verdana;font-size:15px">Basis Size</p>' 
)
startButton = widgets.ToggleButton(
    value = False, 
    description = "Start Calculation",
    button_style = "info"
)
info = widgets.HTML(
    value = '<p style="font-family:verdana;font-size:15px"></p>'
)
massWidget = widgets.Dropdown(
    options = ["Default NIST Value", "Times 2", "Divide by 2"],
    value = "Default NIST Value",
    description = '<p style="font-family:verdana;font-size:15px">Mass AMU</p>'
)
wWidget = widgets.Dropdown(
    options = ["Default NIST Value", "Times 2", "Divide by 2"],
    value = "Default NIST Value",
    description = '<p style="font-family:verdana;font-size:15px">ω<sub>e</sub> 1/cm</p>' 
)


wValues = { "Default NIST Value" : "DEF", 
            "Divide by 2" : 0.5, 
            "Times 2" : 2, 
          }
massValues = { "Default NIST Value" : "DEF", 
                "Divide by 2" : 0.5 , 
                "Times 2" : 2, 
              }

out = widgets.Output()

In [4]:
#main logic 
storagePath = ".\\dataStorage\\" if "win" in sys.platform else "./dataStorage/"

def buildSchrodSolution(schrodFileName):
    dc = getDiatomicConstants(diatomicName.value)
        
    #check if diatomic provided can be found
    if(dc == False):
        info.value = "Could not find requested diatomic, please try again."
        return False
    
    info.value = '<p style="font-family:verdana;font-size:15px">Calculating.....</p>'

    #check if alternative mass or omega e shouuld be used
    if(wValues[wWidget.value] != "DEF"):
        dc["w"] = dc["w"] * wValues[wWidget.value]
    if(massValues[massWidget.value] != "DEF"):
        dc["u"] = dc["u"] * massValues[massWidget.value]
    print(dc)
    
    basis = basisSet(dc, basisFunctionClass = how, size = basisSize.value)
    fitting = morsePotential(dc, rkr(dc).compute())
    schrodSolution = schrod(HOperator(TOperator(basis), VOperator(basis, fitting)), basis, fitting)

    with open(schrodFileName, "wb") as schrodBin:
        dill.dump(schrodSolution, schrodBin)     
        
        
    return schrodSolution

def compute(value):
    
    startButton.value = False
    if(not value["new"]):
        return 
    
    schrodFileName = storagePath + diatomicName.value + str(wValues[wWidget.value]) + str(massValues[massWidget.value])
        
    #check if data storage already has the wavefunctions calculated
    if(os.path.exists(schrodFileName)):
        with open(schrodFileName, 'rb') as schrodBin:
            schrodSolution = dill.load(schrodBin)    
            
            #if the user is running a larger basis set then allow the user to run their calculation
            if(schrodSolution.maxWaveFunctions < basisSize.value):
                endCalcMessage = '<p style="font-family:verdana;font-size:15px">Calculation Completed</p>'
                schrodSolution = buildSchrodSolution(schrodFileName)
                
            else: 
                endCalcMessage = '<p style="font-family:verdana;font-size:15px">Found Saved Diatomic Schrodinger Solution with basis size of ' + str(schrodSolution.basis.size) + '.</p>'
    else: 
        endCalcMessage = '<p style="font-family:verdana;font-size:15px">Calculation Completed</p>'
        schrodSolution = buildSchrodSolution(schrodFileName)
        
    if(schrodSolution == False):
        return
        
    info.value = endCalcMessage

    with out:
        graph = schrodSolution.graph(getGraph = True)
        out.clear_output()
        display(graph)   
    
startButton.observe(compute, names="value")

display(widgets.VBox([
        widgets.HBox([diatomicName, basisSize, startButton]),
        widgets.HBox([wWidget, massWidget]),
        info]))
display(out)

Output()

In [5]:
#NO
#CO
#N2
#HCL
#O2
#HF
#CL2
#Omega 
#Mass